## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-21-20-52-20 +0000,nyt,White House’s East Wing Partially Demolished t...,https://www.nytimes.com/2025/10/20/us/politics...
1,2025-10-21-20-52-00 +0000,wsj,Basic Materials Roundup: Market Talk,https://www.wsj.com/articles/basic-materials-r...
2,2025-10-21-20-51-00 +0000,wsj,The Trump administration is planning to releas...,https://www.wsj.com/politics/policy/trump-admi...
3,2025-10-21-20-50-57 +0000,nyt,"Trump Dismisses Another Inspector General, Fue...",https://www.nytimes.com/2025/10/21/us/politics...
4,2025-10-21-20-48-44 +0000,nyt,‘KPop Demon Hunters’ Helps Lift Netflix Revenu...,https://www.nytimes.com/2025/10/21/business/me...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2370/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,59
1,house,19
90,up,18
0,white,16
119,will,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
2,2025-10-21-20-51-00 +0000,wsj,The Trump administration is planning to releas...,https://www.wsj.com/politics/policy/trump-admi...,173
102,2025-10-21-17-27-00 +0000,wsj,Key Senate Republicans are calling for the wit...,https://www.wsj.com/politics/policy/paul-ingra...,144
326,2025-10-20-22-36-00 +0000,wsj,"President Trump’s $100,000 H-1B visa fee will ...",https://www.wsj.com/politics/policy/u-s-narrow...,133
89,2025-10-21-17-55-57 +0000,nypost,Hillary Clinton takes shot at Trump for White ...,https://nypost.com/2025/10/21/us-news/hillary-...,121
132,2025-10-21-16-24-49 +0000,wapo,White House says no Putin-Trump summit anytime...,https://www.washingtonpost.com/world/2025/10/2...,119


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
2,173,2025-10-21-20-51-00 +0000,wsj,The Trump administration is planning to releas...,https://www.wsj.com/politics/policy/trump-admi...
40,76,2025-10-21-19-53-54 +0000,nyt,Vance Lands in Israel as U.S. Tries to Shore U...,https://www.nytimes.com/2025/10/21/world/middl...
102,75,2025-10-21-17-27-00 +0000,wsj,Key Senate Republicans are calling for the wit...,https://www.wsj.com/politics/policy/paul-ingra...
320,60,2025-10-20-23-00-00 +0000,wsj,Business and political leaders have intensifie...,https://www.wsj.com/politics/elections/nyc-may...
278,51,2025-10-21-04-53-16 +0000,wapo,"Japan elects first female leader, a hawk who p...",https://www.washingtonpost.com/world/2025/10/2...
218,43,2025-10-21-10-05-40 +0000,wapo,Israel IDs more hostages’ bodies as Hamas says...,https://www.washingtonpost.com/world/2025/10/1...
178,41,2025-10-21-13-25-26 +0000,nypost,Former French President Sarkozy begins 5-year ...,https://nypost.com/2025/10/21/world-news/forme...
177,41,2025-10-21-13-32-06 +0000,nyt,"Suspect Charged in Paris Museum Heist (No, Not...",https://www.nytimes.com/2025/10/21/world/europ...
173,41,2025-10-21-13-56-46 +0000,nypost,Warner Bros. Discovery says it’s open to a sal...,https://nypost.com/2025/10/21/business/warner-...
48,34,2025-10-21-19-42-32 +0000,missionlocal,S.F. joins coalition urging Supreme Court to b...,https://missionlocal.org/2025/10/s-f-joins-coa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
